# Graphene Sheet with Interactive Physics Parameters

This notebook simulates a graphene sheet with OpenMM, and exposes parameters that can be controlled in real time.
Along the way, we'll learn:

* How to add restraints to OpenMM simulations 
* How to expose functions as **commands** that can be run remotely 
* How to add sliders and buttons to Jupyter notebooks

## Setup the OpenMM Simulation 

First, we set up an OpenMM simulation of graphene. In this case, we've already generated an OpenMM XML file and have a PDB ready to use for the topology. See the [neuraminidase example](./openmm_neuraminidase.ipynb) for a more detailed look at setting up OpenMM simulations. 

In [ ]:
import simtk.openmm.app as app
import simtk.openmm as mm 
import simtk.unit as unit
pdb_file = app.PDBFile('openmm_files/graphene_with_bonds.pdb')
system_xml = 'openmm_files/graphene_omm.xml'

In [ ]:
#read the system into OpenMM 
with open(system_xml, 'r') as f:
    system_string = f.read()
system: mm.System
system = mm.XmlSerializer.deserialize(system_string)

Great, we've got a PDB topology, and a definition of all of the OpenMM forces to use with it. 

In [ ]:
system.getNumForces(), system.getNumParticles()

In [ ]:
pdb_file.getTopology().getNumAtoms()

For this simulation, we want to hold the corners of the graphene sheet in place. We do that with spring force restraints. 
Narupa has some shortcuts for setting these up with OpenMM.

In [ ]:
from narupa.openmm.potentials import restrain_particles

In [ ]:
atoms_to_restrain = [0, 38, 641, 679] # the corner atoms. 
force = restrain_particles(pdb_file.positions, atoms_to_restrain, 10000 * unit.kilojoule_per_mole / unit.nanometer ** 2)
force_index = system.addForce(force)
print(f"Added force with index {force_index}. System now has {system.getNumForces()} forces.")

Now we create an OpenMM simulation with it. 

In [ ]:
simulation = app.Simulation(pdb_file.topology, system, mm.LangevinIntegrator(300 * unit.kelvin, 1.0/unit.picosecond, 1.0*unit.femtosecond))
simulation.context.setPositions(pdb_file.positions)
simulation.minimizeEnergy()

Run a few steps to make sure it's working

In [ ]:
simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)
simulation.step(1000)

## Run the Narupa Server

We'll use Narupa's OpenMMIMDRunner to simplify running the server

In [ ]:
from narupa.ase.openmm.runner import OpenMMIMDRunner, ImdParams

In [ ]:
runner = OpenMMIMDRunner(simulation, ImdParams(port=0, time_step=0.5, verbose=False))

In [ ]:
print(f'{runner.name}: serving on {runner.address}:{runner.port}')

In [ ]:
runner.run(20)

Let's leave it running in the background 

In [ ]:
runner.run()

## Controlling the Physics From the Notebook

Since we're running the simulation with ASE, we can change the parameters while it's running. 
The cell below sets up some methods for changing the temperature, friction and timestep

In [ ]:
temp_min_val = 0
temp_max_val = 10000
friction_min_val = 0.01
friction_max_val = 100
timestep_min_val = 0.01
timestep_max_val = 1.5
import ase.units as units

def set_temperature(temperature=300):
    """
    Sets the temperature in the ASE simulation.

    :param temperature: Temperature to set, in kelvin.
    """

    if not temp_min_val <= temperature <= temp_max_val:
        raise ValueError(f'Temperature must be in range {temp_min_val} - {temp_max_val} Kelvin.')
    runner.dynamics.set_temperature(temperature * units.kB)


def set_friction(friction=1):
    """
    Sets the friction in the ASE simulation.

    :param friction: Friction, in ASE units * 1000, for visualisation purposes
    """

    if not friction_min_val <= friction <= friction_max_val:
        raise ValueError(f'Friction must be in range {friction_min_val} - {friction_max_val}.')
    runner.dynamics.set_friction(friction / 1000.0)


def set_timestep(timestep=0.5):
    """
    Sets the timestep in the ASE simulation.

    :param timestep: Timestep, in femtoseconds.
    """

    if not timestep_min_val <= timestep <= timestep_max_val:
        raise ValueError(f'Timestep must be in range {timestep_min_val} - {timestep_max_val}')
    timestep = timestep * units.fs
    runner.dynamics.set_timestep(timestep)

Now we set up some sliders and buttons so we can adjust these on the fly in the notebook

In [ ]:
# imports for sliders
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Sliders for temperature, friction and timestep
interact(set_temperature, temperature=(temp_min_val,temp_max_val));
interact(set_friction, friction=(friction_min_val,friction_max_val, 1.0));
interact(set_timestep, timestep=(timestep_min_val,timestep_max_val, 0.01));

# buttons and toggles for playing and reset
reset_button = widgets.Button(description="Restart Simulation")
play_button = widgets.ToggleButton(description="Playing")
output = widgets.Output()
display(reset_button, output)
display(play_button, output)

def on_reset_clicked(b):
    with output:
        runner.imd.reset()

def on_play_clicked(obj):
    with output:
        if obj['new']:  
            runner.imd.play()
        else:
            runner.imd.pause()

reset_button.on_click(on_reset_clicked)
play_button.observe(on_play_clicked, 'value')

Enter the server in VR and see how the dynamics change when you lower the temperature and and massively increase the friction!

## Remote Control Commands

While controlling these parameters from the notebook is pretty cool, doing it from VR or a dedicated application would be even better. 

Narupa provides a mechanism for doing this via *commands*. A command consists of a command name and a handler function to call when the client requests to run a command by that name.

Let's set up our timestep, friction and temperature methods as commands

In [ ]:
# Methods for interacting with the simulation.
TIMESTEP_COMMAND = "sim/timestep"
FRICTION_COMMAND = "sim/friction"
TEMPERATURE_COMMAND = "sim/temperature"

# the following line unregisters the commands if they've already been registered. 
for command in [TIMESTEP_COMMAND, FRICTION_COMMAND, TEMPERATURE_COMMAND]:
    try:
        runner.app_server.server.unregister_command(command)
    except:
        pass

runner.app_server.server.register_command(TIMESTEP_COMMAND, set_timestep)
runner.app_server.server.register_command(TEMPERATURE_COMMAND, set_temperature)
runner.app_server.server.register_command(FRICTION_COMMAND, set_friction)

Now, we can connect a client, and call the commands

In [ ]:
from narupa.app import NarupaImdClient
client =  NarupaImdClient.connect_to_single_server(port=runner.port)    

We can see all the available commands, note that play, pause, reset and step are already registered, as are the ones we've just added 

In [ ]:
commands = client.update_available_commands();
dict(commands).keys()

So now we can set the temperature remotely (try this on another computer!):

In [ ]:
client.run_command('sim/temperature', temperature=200);
# print out the temperature to check it's worked, we have to convert from ASE units to Kelvin
runner.dynamics.temp / units.kB

With this functionality, you could write your own UI in Unity with our [libraries](https://gitlab.com/intangiblerealities/narupa-applications/narupa-imd/-/blob/master/Assets/Plugins/Narupa/Grpc/GrpcClient.cs#L54), a python web app, or even [C++](https://gitlab.com/intangiblerealities/narupatools/narupa-protocol-cpp).

## Gracefully Terminate

In [ ]:
client.close()
runner.close()

## Next Steps

* Explore setting up [commands and synchronizing state](../fundamentals/commands_and_state.ipynb)